In [1]:
import os
import shutil
import random

# Set paths
source_dir = r"D:\Major Project\Rasp\old\Combined"
output_base = r"D:\Major Project\Rasp\old\split_clients_4_with_rounds"
os.makedirs(output_base, exist_ok=True)

# Load image paths
drowsy_images = [os.path.join(source_dir, 'drowsy', f) for f in os.listdir(os.path.join(source_dir, 'drowsy'))]
notdrowsy_images = [os.path.join(source_dir, 'notdrowsy', f) for f in os.listdir(os.path.join(source_dir, 'notdrowsy'))]

# Shuffle
random.shuffle(drowsy_images)
random.shuffle(notdrowsy_images)

# Client distributions (drowsy_count, notdrowsy_count)
clients = {
    'client1': (3144, 786),     # 80-20 drowsy-heavy
    'client2': (786, 3144),     # 80-20 notdrowsy-heavy
    'client3': (2358, 1572),    # 60-40 drowsy-heavy
    'client4': (1572, 2358),    # 60-40 notdrowsy-heavy
}

# Function to copy images into 5 rounds
def split_into_rounds(images, rounds=5):
    size = len(images) // rounds
    return [images[i*size:(i+1)*size] for i in range(rounds)]

# Function to copy images
def copy_images(images, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for img_path in images:
        shutil.copy(img_path, dest_dir)

# Distribute images
for client, (drowsy_count, notdrowsy_count) in clients.items():
    client_dir = os.path.join(output_base, client)

    drowsy_subset = drowsy_images[:drowsy_count]
    notdrowsy_subset = notdrowsy_images[:notdrowsy_count]

    # Remove used samples
    drowsy_images = drowsy_images[drowsy_count:]
    notdrowsy_images = notdrowsy_images[notdrowsy_count:]

    # Split into 5 rounds
    drowsy_rounds = split_into_rounds(drowsy_subset)
    notdrowsy_rounds = split_into_rounds(notdrowsy_subset)

    for round_idx in range(5):
        round_dir = os.path.join(client_dir, f'round{round_idx+1}')
        copy_images(drowsy_rounds[round_idx], os.path.join(round_dir, 'drowsy'))
        copy_images(notdrowsy_rounds[round_idx], os.path.join(round_dir, 'notdrowsy'))


In [2]:
import os
import shutil
import random

# Set paths
source_dir = r"D:\Major Project\Rasp\old\Combined"
output_base = r"D:\Major Project\Rasp\old\split_clients_2_with_rounds"
os.makedirs(output_base, exist_ok=True)

# Load and shuffle image paths
def load_images(label):
    path = os.path.join(source_dir, label)
    return [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

drowsy_images = load_images('drowsy')
notdrowsy_images = load_images('notdrowsy')
random.shuffle(drowsy_images)
random.shuffle(notdrowsy_images)

# Client distributions (drowsy_count, notdrowsy_count)
clients = {
    'client1': (2358, 1572),  # 60% drowsy
    'client2': (1572, 2358),  # 60% notdrowsy
}

# Split function with better handling
def split_into_rounds(images, rounds=5):
    avg = len(images) / float(rounds)
    out, last = [], 0.0
    while last < len(images):
        out.append(images[int(last):int(last + avg)])
        last += avg
    return out

# Copy images
def copy_images(images, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for img_path in images:
        shutil.copy(img_path, dest_dir)

# Distribute to clients
for client, (drowsy_count, notdrowsy_count) in clients.items():
    client_dir = os.path.join(output_base, client)

    assert drowsy_count <= len(drowsy_images), f"Not enough drowsy images for {client}"
    assert notdrowsy_count <= len(notdrowsy_images), f"Not enough notdrowsy images for {client}"

    drowsy_subset = drowsy_images[:drowsy_count]
    notdrowsy_subset = notdrowsy_images[:notdrowsy_count]

    # Update remaining pool
    drowsy_images = drowsy_images[drowsy_count:]
    notdrowsy_images = notdrowsy_images[notdrowsy_count:]

    # Split into rounds
    drowsy_rounds = split_into_rounds(drowsy_subset)
    notdrowsy_rounds = split_into_rounds(notdrowsy_subset)

    for round_idx in range(5):
        round_dir = os.path.join(client_dir, f'round{round_idx+1}')
        copy_images(drowsy_rounds[round_idx], os.path.join(round_dir, 'drowsy'))
        copy_images(notdrowsy_rounds[round_idx], os.path.join(round_dir, 'notdrowsy'))
        print(f"{client} - Round {round_idx+1}: {len(drowsy_rounds[round_idx])} drowsy, {len(notdrowsy_rounds[round_idx])} notdrowsy")


IndentationError: expected an indented block after function definition on line 11 (2984512650.py, line 12)